In [2]:
import quandl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import pandas_datareader.data as web
from datetime import date
import scipy

In [3]:
quandl.ApiConfig.api_key = "CTnhpzRXRtpR9w1bU7ae"

ue_rate = quandl.get("FRED/UNRATE", start_date="2000-12-01")
job_vacancies = quandl.get("FRED/JTSJOR", start_date="2000-12-01")

In [4]:
report_start = date.fromisoformat("2000-12-01")
report_end = date.today()
data = web.get_data_fred(["UNRATE", "JTSJOR"], report_start, report_end)

data["UNRATE"] = data["UNRATE"].apply(lambda x: x / 100)
data["JTSJOR"] = data["JTSJOR"].apply(lambda x: x / 100)

pd.options.display.float_format = lambda x: (
    "{:.1%}".format(x) if x < 100 else "{:.0f}".format(x)
)
data.loc["2001":]

,UNRATE,JTSJOR
DATE,,
2001-01-01,4.2%,3.8%
2001-02-01,4.2%,3.7%
2001-03-01,4.3%,3.5%
2001-04-01,4.4%,3.4%
2001-05-01,4.3%,3.2%
...,...,...
2024-03-01,3.8%,5.0%
2024-04-01,3.9%,4.8%
2024-05-01,4.0%,4.9%


In [5]:
report_start = date.fromisoformat("2000-12-01")
report_end = date.today()
data = web.get_data_fred(['UNRATE', 'JTSJOR'], report_start, report_end)

bvg_curve_df = data.iloc[:, [0, 1]].dropna()
fig, ax = plt.subplot(figsize=(20, 10))
sc = ax.scatter(
    x=bvg_curve_df.iloc[:, 0],
    y=bvg_curve_df.iloc[:, 1],
    c=mdates.date2num(bvg_curve_df),
)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))
ax.set_xlabel(bvg_curve_df.column[0])
ax.set_ylabel(bvg_curve_df.column[1])
ax.set_title("US Beveridge Curve from {:%Y}".format(bvg_curve_df.index[0]))

# Add labels to the last three months for ease of understanding the current trend
for i in range(-1, -4, -1):
    plt.text(
        bvg_curve_df.iloc[i, 0],
        bvg_curve_df.iloc[i, 1],
        f"{bvg_curve_df.index[i]:%Y-%m}",
    )
ax.grid(True)
loc = mdates.AutoDateFormatter()
_ = fig.colorbar(sc, ticks=loc, format=mdates.AutoDateFormatter(loc))

RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=2000-12-01
Response Text:
b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <title>Error - St. Louis Fed</title>\r\n    <meta name="description" content="">\r\n    <meta name="keywords" content="">    \r\n    <link rel="stylesheet" type="text/css" href="/assets/bootstrap/dist/css/bootstrap.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/css/home.min.css?1553087253">\r\n    <link rel="stylesheet" type="text/css" href="/assets/fontawesome-free/css/all.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/assets/select2/dist/css/select2.min.css">\r\n    <style>p {\r\n        margin-bottom: 1.5em;\r\n    }</style>\r\n</head>\r\n<body>\r\n<link rel="preconnect" href="https://fonts.gstatic.com">\n<link href="http